### Aim - Smart Ticket resolution system

#### Section 1 : Exploratory Data Analysis

- begin with a short EDA of provided data files
- inspect meta information associated with old and new tickets
- identify potential fields of interest
- document findings

In [2]:
import re
import string
import os

import pandas as pd
pd.set_option('display.max_colwidth', 500)
import numpy as np

from sentence_transformers import SentenceTransformer
import hnswlib

In [3]:
def read_file(file_path):
    """
    Reads a file (CSV, XLSX, or JSON) into a pandas DataFrame.
    
    Parameters:
        file_path (str): Path to the file.
        
    Returns:
        pd.DataFrame: DataFrame containing the data from the file.
    """
    # Get the file extension
    _, file_extension = os.path.splitext(file_path)
    
    # Read the file based on its extension
    if file_extension == '.csv':
        return pd.read_csv(file_path)
    elif file_extension == '.xlsx':
        return pd.read_excel(file_path)
    elif file_extension == '.json':
        return pd.read_json(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")


csv_data = read_file('../data/old_tickets/ticket_dump_1.csv')
print ("No. of items in csv file: ", csv_data.shape)
xlsx_data = read_file('../data/old_tickets/ticket_dump_2.xlsx')
print ("No. of items in xlsx file: ", xlsx_data.shape)
json_data = read_file('../data/old_tickets/ticket_dump_3.json')
print ("No. of items in json file: ", json_data.shape)

No. of items in csv file:  (10, 8)
No. of items in xlsx file:  (10, 8)
No. of items in json file:  (10, 8)


In [4]:
print (csv_data.info())

csv_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ticket ID    10 non-null     object
 1   Issue        10 non-null     object
 2   Category     10 non-null     object
 3   Resolution   10 non-null     object
 4   Date         10 non-null     object
 5   Agent Name   10 non-null     object
 6   Resolved     10 non-null     bool  
 7   Description  10 non-null     object
dtypes: bool(1), object(7)
memory usage: 698.0+ bytes
None


,Ticket ID,Issue,Category,Resolution,Date,Agent Name,Resolved,Description
0,TCKT-1010,Email synchronization error,Software,Reconfigured email settings,2024-01-20,Alice Johnson,True,"Emails not syncing correctly across devices, causing communication delays. This is a recurring issue."
1,TCKT-1011,VPN disconnection issues,Network,VPN settings updated,2024-01-21,Bob Smith,False,Repeated VPN disconnections impacting remote access to company network. This is a recurring issue.
2,TCKT-1012,Password reset issue,Account Management,Password reset successfully,2024-01-22,Cathy Brown,True,User unable to reset password due to system error. This is a recurring issue.
3,TCKT-1013,Request for software installation rights,Software,Installation permissions granted,2024-01-23,David Clark,False,Employee requesting installation rights for new software. This is a recurring issue.
4,TCKT-1044,Printer connectivity problem,Hardware,Printer driver reinstalled,2024-01-24,Eva Adams,True,Printer not connecting to any devices in the office. This is a recurring issue.


In [5]:
print (json_data.info())

json_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 20 to 29
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Ticket ID    10 non-null     object        
 1   Issue        10 non-null     object        
 2   Category     10 non-null     object        
 3   Resolution   10 non-null     object        
 4   Date         10 non-null     datetime64[ns]
 5   Agent Name   10 non-null     object        
 6   Resolved     10 non-null     bool          
 7   Description  10 non-null     object        
dtypes: bool(1), datetime64[ns](1), object(6)
memory usage: 650.0+ bytes
None


,Ticket ID,Issue,Category,Resolution,Date,Agent Name,Resolved,Description
20,TKT1000,Email account locked,Hardware,Check network settings,2024-01-16 19:30:28,Tammy Cardenas,False,User's email account is locked due to multiple incorrect password attempts. Requires password reset and account unlock.
21,TKT1001,Printer not connecting,Hardware,Update software,2024-01-20 11:43:44,Julie Leonard,True,"Network printer is not connecting to the office network, making it impossible for employees to print documents."
22,TKT1002,VPN connectivity issue,Software,Replace hardware component,2024-01-24 22:03:01,Charlotte Grant,False,"VPN connection is frequently dropping, causing remote work disruptions. User needs a stable VPN connection."
23,TKT1003,Software installation error,Hardware,Replace hardware component,2024-01-13 23:40:20,John Tucker,True,Error encountered during the installation of a critical software update. Installation process fails with an error code.
24,TKT1004,Computer running slow,Network,Update software,2024-01-12 01:26:58,Michele Gonzalez,True,"User's computer is experiencing significant slowdowns, especially when running multiple applications. Might require performance diagnosis."


In [6]:
print (xlsx_data.info())

xlsx_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ticket ID    10 non-null     object
 1   Issue        10 non-null     object
 2   Category     10 non-null     object
 3   Resolution   10 non-null     object
 4   Date         10 non-null     object
 5   Agent Name   10 non-null     object
 6   Resolved     10 non-null     bool  
 7   Description  10 non-null     object
dtypes: bool(1), object(7)
memory usage: 698.0+ bytes
None


,Ticket ID,Issue,Category,Resolution,Date,Agent Name,Resolved,Description
0,TCKT-1050,Email client crashes on opening,Software,Email client reinstalled,2024-01-30,Alice Johnson,True,The email client crashes whenever the user tries to open it.
1,TCKT-1051,VPN not connecting,Network,VPN client updated,2024-01-31,Bob Smith,False,"VPN fails to connect to the company\'s network, showing error messages."
2,TCKT-1052,Account locked out - needs reset,Account Management,Account unlocked and password reset,2024-02-01,Cathy Brown,True,User\'s account is locked out after multiple wrong password attempts.
3,TCKT-1053,Permission needed for software update,Software,Software update permissions granted,2024-02-02,David Clark,False,"Update required for a key software, but user lacks installation rights."
4,TCKT-1054,Printer printing blank pages,Hardware,Printer maintenance performed,2024-02-03,Eva Adams,True,The printer is only printing blank pages despite having ink.


In [9]:
# concat all this into a single dataframe
resolved_tickets_df = pd.concat([csv_data, xlsx_data, json_data], ignore_index=True)

print("Combined resolved tickets dataset items length: ", resolved_tickets_df.shape)


# get unique values in specific columns
print ("Unique values in 'Category' column:", resolved_tickets_df['Category'].unique())

Combined resolved tickets dataset items length:  (30, 8)
Unique values in 'Category' column: ['Software' 'Network' 'Account Management' 'Hardware']


In [10]:
resolved_tickets_df.head()

,Ticket ID,Issue,Category,Resolution,Date,Agent Name,Resolved,Description
0,TCKT-1010,Email synchronization error,Software,Reconfigured email settings,2024-01-20,Alice Johnson,True,"Emails not syncing correctly across devices, causing communication delays. This is a recurring issue."
1,TCKT-1011,VPN disconnection issues,Network,VPN settings updated,2024-01-21,Bob Smith,False,Repeated VPN disconnections impacting remote access to company network. This is a recurring issue.
2,TCKT-1012,Password reset issue,Account Management,Password reset successfully,2024-01-22,Cathy Brown,True,User unable to reset password due to system error. This is a recurring issue.
3,TCKT-1013,Request for software installation rights,Software,Installation permissions granted,2024-01-23,David Clark,False,Employee requesting installation rights for new software. This is a recurring issue.
4,TCKT-1044,Printer connectivity problem,Hardware,Printer driver reinstalled,2024-01-24,Eva Adams,True,Printer not connecting to any devices in the office. This is a recurring issue.


In [12]:
# short EDA to check the length of content in "Description" column
# to check if NLP pre-processing steps add further value or NOT

# 1. Calculate Text Lengths:
df_desc = pd.DataFrame()
df_desc['Description_Length'] = resolved_tickets_df['Description'].str.len()

# 2. Calculate Statistics:
min_length = df_desc['Description_Length'].min()
max_length = df_desc['Description_Length'].max()
average_length = df_desc['Description_Length'].mean()

# 3. Display Results:
print(f"Minimum Length: {min_length}")
print(f"Maximum Length: {max_length}")
print(f"Average Length: {average_length}")

# 1. Get Indices of Min and Max Lengths:
min_index = df_desc['Description_Length'].idxmin()  #Index of the minimum length
max_index = df_desc['Description_Length'].idxmax()  #Index of the maximum length

# 2. Access and Print the Descriptions:
shortest_description = resolved_tickets_df.loc[min_index, 'Description']
longest_description = resolved_tickets_df.loc[max_index, 'Description']

print(f"Shortest Description:\n{shortest_description}")
print(f"\nLongest Description:\n{longest_description}")

Minimum Length: 51
Maximum Length: 141
Average Length: 89.5
Shortest Description:
The PC is experiencing random restarts during work.

Longest Description:
The office Wi-Fi network keeps disconnecting, affecting multiple employees. Could be an issue with the Wi-Fi router or network configuration.


In [13]:
# check the test data --> new tickets

new_tickets_df = pd.read_csv('../data/new_tickets.csv')

print ("No. of items in new_tickets test file: ", new_tickets_df.shape)
new_tickets_df.info()

No. of items in new_tickets test file:  (10, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ticket ID    10 non-null     object
 1   Issue        10 non-null     object
 2   Description  10 non-null     object
 3   Category     10 non-null     object
 4   Date         10 non-null     object
dtypes: object(5)
memory usage: 528.0+ bytes


In [14]:
new_tickets_df.head()

,Ticket ID,Issue,Description,Category,Date
0,TCKT-2000,VPN connection timeout,VPN connection times out frequently during use.,Network,2024-02-09
1,TCKT-2001,Emails not syncing on mobile,User's emails are not syncing properly on their mobile device.,Software,2024-02-10
2,TCKT-2002,New software installation request,A request to install new project management software.,Software,2024-02-11
3,TCKT-2003,Laptop screen flickering,The laptop screen starts flickering intermittently during use.,Hardware,2024-02-12
4,TCKT-2004,Cannot open files in shared drive,User reports an error when trying to open files in the shared drive.,Network,2024-02-13


----

### Key Observations from EDA  

#### 1. Dataset Overview  

The dataset consists of resolved and new tickets stored across different file formats: **CSV, XLSX, and JSON**.  

##### **Resolved Tickets Dataset**  
- Contains **30 resolved tickets** across three files:  
  - `ticket_dump_1.csv`  
  - `ticket_dump_2.xlsx`  
  - `ticket_dump_3.json`  
- Each ticket has **8 fields**:  
  - `Ticket ID`, `Issue`, `Category`, `Resolution`, `Date`, `Agent Name`, `Resolved`, `Description`.  
- All three files share a **consistent structure**.  

##### **New Tickets Dataset**  
- Contains **10 new tickets** stored in `new_tickets.csv`.  
- Each ticket has **5 fields**:  
  - `Ticket ID`, `Issue`, `Description`, `Category`, `Date`.  

### 2. Data Structure  

- Columns: `Ticket ID`, `Issue`, `Category`, `Resolution`, `Date`, `Agent Name`, `Resolved`, `Description`.  
- Data types: Mostly **categorical** (`object`), with `Resolved` as a **boolean**.  

### 3. Missing Data  

- **No missing values** were observed in the dataset.  

### 4. Key Insights  

- Each ticket has a **unique `Ticket ID`**.  
- Issues are categorized under `Category`, which can help in **classification**.  
- `Resolution` contains useful text that could be analyzed for **resolution patterns**.  
- `Resolved` is a **boolean field** indicating whether the ticket was **closed** or **still open**.  

### 5. Key Fields of Interest  

#### **Issue**  
- Describes the **problem** reported by the user.  

#### **Category**  
- Groups tickets into predefined categories such as:  
  - **Software**  
  - **Hardware**  
  - **Network**  
  - **Account Management**  

#### **Resolution**  
- Provides the **solution** or steps taken to resolve the issue.  

#### **Description**  
- Offers a **detailed explanation** of the issue, often including additional **context or symptoms**.  
- **Length Statistics:**  
  - **Minimum:** 51 characters  
  - **Maximum:** 141 characters  
  - **Average:** 89.5 characters  

#### **Resolved**  
- A **boolean flag** indicating whether the ticket was **resolved (`True`)** or **unresolved (`False`)**.  


-----

#### Next Steps

- Core Solution
    - Candidate extraction module
        - match new ticket to K nearest old ticket(s)
        - utilize semantic/vector search for above task with a K Nearest Neighbor (KNN) lookup index
    - Response generation module
        - once identified, gather ticket resolution steps from the KNN tickets
        - formulate a response in natural language to assist human agent in resolution of task
    - Presentation layer
      - Streamlit application to encapsualte the functionality in a web application 

- Performance Evaluation
      - use test dataset for evaluation
      
- document findings

-----
EOF